### Building the Graph

在这个例子中 我们需要构建5块内容：

1）variable： 用于存储两个变量
   训练次数 （不参与训练）
   output
   
2）transformation： 对输入参数进行变形
   采用 placeholder

3）update： 更新的规律

4）summary 用于 tensor board scalars

5）global_operations 
   初始化参数
   merge summary

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
my_graph = tf.Graph()

In [4]:
with my_graph.as_default():
    with tf.name_scope('variables'):
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False)
        total_outputs = tf.Variable(0.0,dtype=tf.float32, trainable=False)
    with tf.name_scope('transformation'):
        with tf.name_scope('input'):
            a = tf.placeholder(tf.float32,shape=(None), name='input_pla_a')
        
        with tf.name_scope('middle'):
            b = tf.reduce_prod(a, name='prouduction_b')
            c = tf.reduce_sum(a, name='sum_c')
        
        with tf.name_scope('output'):
            output = tf.add(b,c, name='output')
    with tf.name_scope('update'):
        updata_total = total_outputs.assign_add(output)
        total_step = global_step.assign_add(1)
    
    with tf.name_scope('summaries'):
        average = tf.div(updata_total, tf.cast(total_step, tf.float32),name='average')
        tf.summary.scalar('out_summary', output)
        tf.summary.scalar('total_summary',updata_total)
        tf.summary.scalar('average_summary',average)
    with tf.name_scope('global_operations'):
        init = tf.global_variables_initializer()
        merged_summaries = tf.summary.merge_all()

### Running the Graph : Session

In [5]:
sess = tf.Session(graph=my_graph)
summary_writer = tf.summary.FileWriter('./example_1', sess.graph)

In [6]:
sess.run(init)

In [15]:
def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    outputs, step, summary = sess.run([output, total_step, merged_summaries], feed_dict=feed_dict)
    #print('step=', step, 'summary=', summary)
    print(outputs)
    print(step)
    summary_writer.add_summary(summary, global_step=step)
    summary_writer.add_graph(sess.graph)

In [16]:
run_graph([2,8])
run_graph([3,11,1])
run_graph([5])
run_graph([2,3,99,9])
run_graph([1,0,0])
run_graph([[1],[2],[0]])

26.0
19
48.0
20
10.0
21
5459.0
22
1.0
23
3.0
24


In [12]:
summary_writer.flush()

In [17]:
summary_writer.close()
sess.close()

In [1]:
!tensorboard --logdir = './example_l'

'tensorboard' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
